# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests as req
import re

url = 'https://en.wikipedia.org/wiki/Data_science'

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
data=req.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
soup=bs(data,"lxml")
link=soup.find_all("a")
link[3]

<a href="/wiki/Information_science" title="Information science">information science</a>

In [5]:
wiki=[]
for l in link:
    wiki.append(l.get('href'))
#wiki    

In [6]:
links=[]
for w in wiki:
    l=re.search("^https.+",str(w))
    if l:
        links.append(l.group())
               
links[:10] 

['https://arxiv.org/list/cs.LG/recent',
 'https://en.wikipedia.org/w/index.php?title=Template:Machine_learning_bar&action=edit',
 'https://doi.org/10.1145%2F2500499',
 'https://web.archive.org/web/20141109113411/http://cacm.acm.org/magazines/2013/12/169933-data-science-and-prediction/fulltext',
 'https://web.archive.org/web/20140102194117/http://simplystatistics.org/2013/12/12/the-key-word-in-data-science-is-not-data-it-is-science/',
 'https://www.springer.com/book/9784431702085',
 'https://doi.org/10.1007%2F978-4-431-65950-1_3',
 'https://books.google.com/books?id=oGs_AQAAIAAJ',
 'https://web.archive.org/web/20170320193019/https://books.google.com/books?id=oGs_AQAAIAAJ',
 'https://doi.org/10.1126%2Fscience.1170411']

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
domain = 'http://wikipedia.org'

In [8]:
resp=req.get(domain).content
soup=bs(resp,"lxml")
link=soup.find_all("a")

In [9]:
all_links=[]
for l in link:
    all_links.append(l.get("href"))
#all_links

In [10]:
absolute_l=[]
for l in all_links:
    w=re.search("^https.+",str(l)) 
    if w: 
        absolute_l.append(w.group())
               
absolute_l

['https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications',
 'https://creativecommons.org/licenses/by-sa/3.0/']

In [11]:
relative_l=[]
for l in all_links:
    w=re.search("^/.+",str(l)) 
    if w: 
        relative_l.append(w.group())
               
relative_l[:5]

['//en.wikipedia.org/',
 '//es.wikipedia.org/',
 '//ja.wikipedia.org/',
 '//de.wikipedia.org/',
 '//fr.wikipedia.org/']

In [12]:
x=[r for r in relative_l if bool(re.search("\%",str(r)))==False]
x[:5]

['//en.wikipedia.org/',
 '//es.wikipedia.org/',
 '//ja.wikipedia.org/',
 '//de.wikipedia.org/',
 '//fr.wikipedia.org/']

In [13]:
z=[]
for l in x:
    a="https:"+str(l)
    z.append(a)
z[:5]  

['https://en.wikipedia.org/',
 'https://es.wikipedia.org/',
 'https://ja.wikipedia.org/',
 'https://de.wikipedia.org/',
 'https://fr.wikipedia.org/']

In [14]:
combine=absolute_l+z
combine=set(combine)
combine=list(combine)
combine[:5]


['https://udm.wikipedia.org/',
 'https://xal.wikipedia.org/',
 'https://azb.wikipedia.org/',
 'https://sd.wikipedia.org/',
 'https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications']

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [19]:
import os
os.chdir("wikipedia")

In [20]:
!pwd

/home/david/datamex0320/module-1/lab-parallelization/your-code/wikipedia


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [21]:
from slugify import slugify

In [22]:
def index_page(link):
    try:
        resp = req.get(link).content
        soup = bs(resp,'lxml')
        nombre = str(link)+".html"
        nombre = nombre.replace("/","-")
        with open(nombre,"w+") as f:
            s = f.write(str(soup))
        return s
    except: pass
index_page("https://en.wikipedia.org/wiki/Data_science")

114802

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [23]:
%%time
for e in combine:
    index_page(e)

CPU times: user 29.4 s, sys: 473 ms, total: 29.9 s
Wall time: 2min 7s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [26]:
import multiprocessing as mp

In [27]:
mp.cpu_count()

8

In [28]:
%%time
pool=multiprocessing.Pool()
result=pool.map(index_page, combine)
pool.close()

CPU times: user 21.7 ms, sys: 28.3 ms, total: 50 ms
Wall time: 16.1 s
